# 공공자전거 데이터 분석을 통한 자전거 재배치 최소화

## 개요


**조 분류**: 12조

**조원**: 201904008 곽재원 202104216 백종민

<br>

저희 조는 날씨와 대여 이력 간의 관계, 대여소의 위치와 대여 이력 간의 관계를 통해 "최적의 공공자전거 재배치 및 동선 설계"을 목표로 두고 있습니다.

**목차는 다음과 같습니다.**

1. 분석 목적 및 가설 설정

2. 데이터 불러오기

3. 데이터 전처리

4. 데이터 분석하기

5. 결론 도출

## 1. 분석 목적 및 가설 설정

"최적의 공공자전거 재배치 및 동선 설계"를 수행하기 위해 다음 사항들을 알아보고자 합니다. 

1. 날씨와 공공자전거 이용률간에 상관관계가 있는가.

2. 최적의 재배치 시간은 언제인가.

3. 최적의 재배치 수량은 몇 개인가.

4. 최적의 재배치 경로는 어떻게 구성할 수 있는가?

In [ ]:
# 중간보고서 전반에 걸쳐 공통적으로 사용되는 라이브러리들을 로드한다.
import pandas as pd
import numpy as np
import os

# 경로를 설정하고 해당 작업환경으로 이동한다.
work_dir = '/Users/jaewone/Downloads/공공자전거'
data_path = os.path.join(work_dir, 'data')
location_info_path = os.path.join(data_path, 'locationInfo')
preprocessing_path = os.path.join(data_path, 'preprocessing')
os.chdir(work_dir)

# 폴더가 없을 경우 생성한다.
if not os.path.exists(location_info_path):
    os.mkdir(location_info_path)
if not os.path.exists(preprocessing_path):
    os.mkdir(preprocessing_path)

## 2. 데이터 불러오기



위 분석 목적 및 가설에 대하여 알아보고자 아래 3개의 데이터 세트를 사용할 것입니다. 

- **서울시 공공자전거 대여 이력**

- **서울시 공공자전거 대여소**

- **종관기상관측(ASOS) 자료**

차례로 각각의 데이터를 불러오겠습니다.


### 2.1 "서울시 공공자전거 대여 이력" 정보 수집



본 데이터 세트의 https://data.seoul.go.kr/dataList/OA-15182/F/1/datasetView.do 홈페이지에서 CSV 파일을 다운로드 후 사용할 수 있습니다.

가장 최신 6개월의 자료인 2022년 7월 ~ 2022년 12월 공공자전거 대여 이력 정보를 내려받아 받겠습니다.

이를 통해 가져온 CSV는 다음과 같습니다.

> (가져온 CSV는 data/대여이력 폴더 내부에 저장됩니다.)


In [ ]:
rent_info = pd.read_csv(
    'data/대여이력/서울특별시 공공자전거 대여이력 정보_2212.csv', encoding='cp949')
rent_info.head(5)

### 2.2 "서울시 공공자전거 대여소" 정보 수집



> 본 자료는 API 형식으로 제공됨으로 자료를 내려 받아 csv 파일로 변환한 다음 사용하겠습니다.

본 자료에 대한 API는 https://data.seoul.go.kr/dataList/OA-21235/S/1/datasetView.do 을 통해 발급받을 수 있습니다.

위 홈페이지에서 API key를 발급받은 후 http://openapi.seoul.go.kr:8088/786c4b44666a616538385477496f59/json/bikeStationMaster/1/1000 와 같이 요청하여 사용할 수 있습니다. 이때 마지막 부분의 1/1000에서 1은 시작 페이지 번호이고, 1000은 끝 페이지 번호입니다.

한 번의 요청으로 최대 1,000개의 데이터만 받을 수 있기 때문에
1,000개 단위로 API를 요청하여 정보를 받아온 다음 하나의 CSV 파일로 합쳐주는 작업을 아래 코드를 통해 수행하겠습니다.

> CSV 파일은 data/대여소 정보.csv 라는 파일명으로 저장될 것입니다.


In [ ]:
from requests import get
from csv import DictWriter

# 데이터의 key값들
keys = ['LENDPLACE_ID', 'STATN_ADDR1', 'STATN_ADDR2', 'STATN_LAT', 'STATN_LNT']
row_list = []

page_count = 1
while (True):
    # 요청 url 설정
    url = f'http://openapi.seoul.go.kr:8088/786c4b44666a616538385477496f59/json/bikeStationMaster/{(page_count-1)*1000 + 1}/{page_count * 1000}'

    # OpenAPI에 요청
    print("Request: " + url)
    response = get(url)
    response.encoding = 'utf-8'
    dic = response.json()

    # 마지막 페이지일 경우 반복문을 종료한다.
    if (dic.get('bikeStationMaster') == None and dic.get('RESULT').get('CODE') == 'INFO-200'):
        break

    # 알맞은 데이터를 받았는지 확인
    status = dic['bikeStationMaster']['RESULT']['CODE']
    if (status != 'INFO-000'):
        print(f'Error: Status with code {status}')
        exit(1)

    # csv에 저장할 필요한 정보 추출(요청상태와 같이 불필요한 데이터는 csv에 저장하지 않는다)
    data = dic['bikeStationMaster']['row']
    row_list = row_list + data
    page_count = page_count + 1

# 추출된 정보들을 csv 파일로 저장
save_csv_path = 'data/대여소 정보.csv'
with open(save_csv_path, 'w', encoding='UTF-8', newline='') as f:
    print("Saving with csv...")
    w = DictWriter(f, keys)
    w.writeheader()
    w.writerows(row_list)

print(f'Save file: "{save_csv_path}"')

API를 통해 가져온 정보를 확인해 보겠습니다.

In [ ]:
location = pd.read_csv('data/대여소 정보.csv', encoding='UTF-8')
location.head(5)

### 2.3 "종관기상관측(ASOS)" 자료 수집



본 데이터 세트의 https://data.kma.go.kr/data/grnd/selectAsosRltmList.do?pgmNo=36&tabNo=1 홈페이지에서 CSV 파일을 내려받은 후 사용할 수 있습니다.

서울시의 2022년 각 날짜에 따른 날씨 데이터로 선택하여 내려받았습니다.

이를 통해 가져온 CSV는 다음과 같습니다.

> (가져온 CSV는 data/seoul_weather.csv 로 저장됩니다.)


In [ ]:
weather = pd.read_csv('data/seoul_weather_2022.csv', encoding='cp949')
weather.head(5)

## 3. 데이터 전처리



대여소에 대한 데이터와 대여 이력에 대한 정보 모두 상당히 많은 양의 정보를 담고 있으나 **최적의 자전거 재배치 동선 설계**에는 사용되지 않는 **불필요한** 정보들이 포함되어 있습니다. 또한 정제되지 않은 순수 데이터로 Na, 이상치, 등 정확한 데이터 분석을 방해하는 요소들도 많이 있습니다.

본 단계에서는 불필요한 열의 병합 또는 제거, Na값 처리, 이상치 제거, 등의 방법을 통해 데이터를 전처리한 다음 데이터 분석을 수행할 때 바로 활용할 수 있도록 새로운 CSV 파일로 저장할 것입니다.

### 3.1 "서울시 공공자전거 대여 이력 정보" 전처리



"서울시 공공자전거 대여 이력 정보"에 대한 데이터들은 아래 과정을 통해 전처리를 수행하겠습니다.

1. **데이터 분할**
    
    본 데이터셋은 각 이용에 따른 대여 이력 정보와 대여소 정보를 모두 포함하고 있습니다. 
  
    대여 이력 정보를 통해 최적의 재배치 수량 설계 및 동선 설계에 대여소에 대한 정보는 불필요하니 대여소에 대한 정보를 추출하여 별도의 파일로 분리하겠습니다.


2. **사용하지 않는 열 제거**

    본 분석 목적에 부합하지 않는 열은 메모리 낭비를 불러올 수 있어 제거합니다.


3. **이상치 제거**

    이용시간이 0보다 작거나 24시간(1440)보다 큰 값은 명백한 기계 오류, 등에 의해 발생된 이상치입니다. 이상치를 적절히 제거하여 데이터의 품질을 유지합니다.


4. **csv로 저장**

    전처리가 완료된 데이터프레임을 분석을 수행할 때 부로 사용할 수 있도록 csv로 저장해주겠습니다.


5. **각 파일에 적용**

    "서울시 공공자전거 대여 이력 정보"는 7월부터 12월까지 총 6개의 파일로 구성되어 있습니다. 

    이에 반복문을 통해 위 1번부터 4번까지의 과정을 나머지 데이터셋에 대하여 적용하겠습니다.

#### 3.1.1 데이터 분할



open API를 통해 받아온 "대여소 정보"에는 각 대여소에 주소와 위치값은 존재하지만, 대여소 이름, 대여소 번호, 대여소 거치대 개수와 같은 세부적인 정보를 불포함하고 있습니다.

공공데이터 포털에서 "대여소의 세부적인 정보"를 포함하고 있는 정보를 찾아보았으나 대여소의 위치(위도, 경도)와 세부 정보를 동시에 제공하지 않았습니다.

이에 "서울시 공공자전거 대여 이력 정보" 파일에 있는 대여소 번호, 대여소명, 대여소 거치대 개수를 추출하여 "대여소 정보" 파일에 합쳐주도록 하겠습니다. 본 과정은 아래 순서로 진행됩니다.

1. 대여이력 CSV 파일을 읽어와 "대여대여소ID", "대여 대여소번호", "대여 대여소명", "대여거치대" 열의 값을 가져옵니다.

2. 각 열의 이름을 '대여소ID', '대여소번호', '대여소명', '거치대'로 변경합니다.

3. 대여이력 CSV 파일을 읽어와 "반납반납소ID", "반납 반납소번호", "반납 반납소명", "반납거치대" 열의 값을 가져옵니다.

4. 각 열의 이름을 '대여소ID', '대여소번호', '대여소명', '거치대'로 변경합니다.

5. 위의 두 데이터프레임을 하나로 통합합니다. 통합할 때 대여소ID가 중복되는 값들은 제거합니다.

6. 대여소ID가 결측값을 제거한 뒤 "대여소ID"를 인덱스로 설정하여 빠르게 값을 찾을 수 있도록 합니다.

7. locationInfo 폴더에 하나의 CSV 파일로 저장한다. 파일명은 2212.csv 와 같이 년도와 월로 합니다.

8. 위 과정을 모든 대여이력 CSV 파일에 대해 수행합니다.

9. locationInfo 폴더에 생성된 모든 대여소 세부 정보 CSV 파일들을 불러옵니다.

10. 대여소ID값이 중복될 경우 제거합니다.

11. "대여소 정보.csv" 파일을 가져와 대여소ID값을 기준으로 대여소 세부 정보들을 결합(join)합니다.

12. 변경된 정보를 파일에 덮어씁니다.

8번 이후의 과정은 모든 파일에 대해 처리를 완료한 다음 진행해야 하니 우선 하나의 파일에 대해서 7번 과정까지 진행해 보겠습니다.

> 대여이력 csv파일은 결측값에 대해 "\N" 문자열이 입력되어 있습니다(혹은 pandas가 CSV를 읽어올 때 결측값에 대해 줄내림문자(\N)로 인식했을 수도 있습니다). 이에 \N 값을 Na로 변환하여 제거하겠습니다.


In [ ]:
# 대여이력 정보를 불러온다. 12월 자료에 대해 우선적으로 적용한 다음 다른 자료에도 순차적으로 적용하겠다.
rent_history_data = pd.read_csv(
    'data/대여이력/서울특별시 공공자전거 대여이력 정보_2212.csv', encoding='cp949')

# 필요한 열의 데이터만 가져온다.
location_data = rent_history_data.loc[:, [
    '대여대여소ID', '대여 대여소번호', '대여 대여소명', '대여거치대', '반납대여소ID', '반납대여소번호', '반납대여소명', '반납거치대']]

# 대여소ID에 따른 대여소 정보를 가져온 뒤 열 이름을 통일해준다.
rent_location = location_data[['대여대여소ID', '대여 대여소번호', '대여 대여소명', '대여거치대']]
return_location = location_data[['반납대여소ID', '반납대여소번호', '반납대여소명', '반납거치대']]
rent_location.columns = return_location.columns = ['대여소ID', '대여소번호', '대여소명', '거치대']

# \N값 제거, 결측값 제거, 대여소ID 중복값 제거, 대여소ID를 인덱스로 변환하는 작업을 수행한다.
location_info = (pd.concat([rent_location, return_location])
                 .replace(r"\\N", np.nan, regex=True)
                 .dropna(subset=['대여소ID'])
                 .drop_duplicates(subset="대여소ID")
                 .set_index("대여소ID"))

# csv로 저장
location_info.to_csv('data/locationInfo/2212.csv', encoding='UTF-8-sig')
print("Done")

저장된 CSV는 다음과 같습니다.

> 거치대 숫자가 0인 것은 정상적인 정보가 아니지만 공공자전거는 거치대의 개수와 관계없이 위치기반으로 반납을 처리하며 자전거를 반납할 때 항상 거치대에 거치를 이용하는 것이 아니기에 해당 오류는 무시합니다.


In [ ]:
location_info = pd.read_csv('data/locationInfo/2212.csv', encoding='UTF-8-sig')
location_info.head(5)

#### 3.1.2 사용하지 않는 열 제거



- 본 연구의 목적은 "최적의 자전거 재배치 동선 설계"로 생년월일, 성별, 이용자 종류, 등 이용자에 대한 세부정보는 사용되지 않습니다. 따라서 해당 열들을 제거해 주겠습니다.

- "데여소 정보"에 통합하기 위해 별도의 CSV 파일로 분리하였던 '대여 대여소번호', '대여 대여소명', '대여거치대', '반납대여소번호', '반납대여소명', '반납거치대' 열 또한 제거합니다.

- 각 자전거에 따른 구분을 하는 것이 아니기에 "자전거 번호" 열 또한 제거합니다.

위 과정은 결론적으로 대여이력 정보에서 '대여일시', '반납일시', '이용시간(분)', '이용거리(M)', '대여대여소ID', '반납대여소ID' 열만 가져오는 것과 동일합니다. 이에 아래와 같이 작성될 수 있습니다.


In [ ]:
use_data = rent_history_data.loc[:, [
    '대여일시', '반납일시', '이용시간(분)', '이용거리(M)', '대여대여소ID', '반납대여소ID']]

use_data.info()
use_data.head(5)

#### 3.1.3 이상치 제거



정상적이지 않은 값들은 정확한 데이터 분석을 방해합니다. 따라서 아래 과정을 통해 이상치를 제거하겠습니다.

1. 본 데이터셋의 모든 값은 필수적으로 존재해야 함으로 NA값을 모두 제거하겠습니다.

2. 이용시간이 0보다 작거나 1440(24시간)보다 클 경우 제거합니다.

3. 이용거리가 0보다 작거나 36780(서울 가로 횡단 거리)보다 클 경우 제거합니다.

4. 이동속도가 사이클 대회 선수들의 평균 분속인 약 666(m/min) 보다 큰 것은 GPS 오류, 자동차에 적재하여 이동, 등 정상적인 값이 아니라고 판단하여 제거합니다.<br>이후 처리가 완료된 speed 열은 제거합니다.

> 본 데이터셋(12월)에 대해서는 변경된 데이터를 기존 데이터에 덮어쓰는 방식으로 코드를 작성하였습니다.<br> 하지만 이후 OS 모듈을 이용한 모든 파일에 적용할 때는 파이프라인(pipe function) 함수를 이용하여 작성하였습니다.


**NA값 제거**

본 데이터셋의 모든 값은 필수적으로 존재해야 함으로 NA값을 모두 제거하겠습니다.


In [ ]:
# 1. na값을 모두 제거한다.
use_data = use_data.dropna()

**이용시간 이상치 제거**

이용 시간은 하루 단위로 측정됨으로 0 초과 24 이하의 값을 가져야 합니다.

따라서 이용 시간이 0 이하인 경우와 24시간(1,440분) 초과면 이상치로 판단하여 제거합니다.


In [ ]:
# 2. 이용시간이 0이하, 1440초과인 값을 가지면 제거한다.
use_data = use_data[use_data['이용시간(분)'].between(1, 1440)]

**이용거리 이상치 제거**

서울시 공공자전거 사업은 서울시에서 운영하는 공유 정책 사업으로 공공자전거 대여소는 서울시 지역 내부에만 존재합니다.

따라서 이용거리가 0이하이거나 서울의 가로 횡단 길이에 해당하는 36780m 초과이면 이상치라고 판단하여 제거합니다.


In [ ]:
# 3. 이용거리가 0 이하, 36780 초과인 값을 가지면 제거한다.
use_data = use_data[use_data['이용거리(M)'].between(1, 36780)]

**이동속도 이상치 제거**

이동속도가 사이클 대회 선수들의 평균 분속인 약 666(m/min) 보다 큰 것은 GPS 오류, 자동차에 적재하여 이동, 등 정상적인 값이 아니라고 판단하여 제거합니다.

작업을 수행한 이후, 처리가 완료된 speed 열은 제거합니다.


In [ ]:
# 4. 이동속도가 사이클 대회 선수들의 평균 분속인 약 666(m/min) 보다 큰 것은 GPS 오류, 자동차에 적재하여 이동, 등 정상적인 값이 아니라고 판단하여 제거한다.
# 속도(m/min)를 구하고 계산 결과 발생한 inf 값에 대해 제거한다.
use_data = use_data.assign(speed=use_data['이용거리(M)'] / use_data['이용시간(분)'])
use_data = use_data.replace([np.inf, -np.inf], np.nan).dropna()
use_data = use_data[use_data.speed < 666]

# 처리가 완료된 speed 열은 제거한다.
use_data = use_data.drop(columns='speed')

#### 3.1.4 CSV로 저장



위 과정을 통해 전처리를 완료한 데이터프레임은 데이터 사용시 바로 사용할 수 있도록 csv 파일로 저장합니다.


In [ ]:
# csv 파일로 저장한다.
use_data.to_csv(f'data/preprocessing/2212.csv', index=False)
print(f'Save: data/preprocessing/2212.csv\n')

#### 3.1.5 각 파일에 적용



위 "서울시 공공자전거 대여 이력 정보" 전처리 과정을 각각의 파일에 대해 수행하였습니다.

In [ ]:
def preprocess_csv(file_name, save_file_name, data_path):
    data = pd.read_csv(f'{data_path}/대여이력/{file_name}', encoding='cp949')

    # 필요한 열의 데이터만 가져온다.
    location_data = data.loc[:, ['대여대여소ID', '대여 대여소번호', '대여 대여소명',
                                 '대여거치대', '반납대여소ID', '반납대여소번호', '반납대여소명', '반납거치대']]

    # 대여소ID에 따른 대여소 정보를 가져온 뒤 열 이름을 통일해준다.
    rent_location = location_data[['대여대여소ID', '대여 대여소번호', '대여 대여소명', '대여거치대']]
    return_location = location_data[['반납대여소ID', '반납대여소번호', '반납대여소명', '반납거치대']]
    rent_location.columns = return_location.columns = ['대여소ID', '대여소번호', '대여소명', '거치대']

    # \N값 제거, 결측값 제거, 대여소ID 중복값 제거, 대여소ID를 인덱스로 변환하는 작업을 수행한다.
    location_info = (pd.concat([rent_location, return_location])
                     .replace(r"\\N", np.nan, regex=True)
                     .dropna(subset=['대여소ID'])
                     .drop_duplicates(subset="대여소ID")
                     .set_index("대여소ID"))

    # csv로 저장
    location_info.to_csv(
        f'{data_path}/locationInfo/{save_file_name}.csv', encoding='UTF-8-sig')
    print(f'Save: {data_path}/locationInfo/{save_file_name}.csv')

    use_data = data.loc[:, ['대여일시', '반납일시',
                            '이용시간(분)', '이용거리(M)', '대여대여소ID', '반납대여소ID']]

    use_data = (
        # 1. na값을 모두 제거한다.
        use_data.dropna()

        # 2. 이용시간이 0이하, 1440초과인 값을 가지면 제거한다.
        .pipe(lambda df: df[df['이용시간(분)'].between(1, 1440)])

        # 3. 이용거리가 0 이하, 36780 초과인 값을 가지면 제거한다.
        .pipe(lambda df: df[df['이용거리(M)'].between(1, 36780)])

        # 4. 이동속도가 사이클 대회 선수들의 평균 분속인 약 666(m/min) 보다 큰 것은 GPS 오류, 자동차에 적재하여 이동, 등 정상적인 값이 아니라고 판단하여 제거한다.
        # 속도(m/min)를 구하고 계산 결과 발생한 inf 값에 대해 제거한다.
        .pipe(lambda df: df.assign(speed=df['이용거리(M)'] / df['이용시간(분)']))
        .pipe(lambda df: df.replace([np.inf, -np.inf], np.nan).dropna())
        .pipe(lambda df: df[df.speed < 666])

        # 처리가 완료된 speed 열은 제거한다.
        .pipe(lambda df: df.drop(columns='speed'))
    )

    # csv 파일로 저장한다.
    use_data.to_csv(f'{data_path}/preprocessing/{save_file_name}.csv', index=False)
    print(f'Save: {data_path}/preprocessing/{save_file_name}.csv\n')


# 사용할 파일들을 리스트업한다.
file_list = []
file_dir = os.path.join(data_path, '대여이력')
for file_name in os.listdir(file_dir):
    if (os.path.splitext(file_name)[1] == '.csv'):
        file_list.append(file_name)

# 폴더가 없을 경우 생성한다.
if not os.path.exists(location_info_path):
    os.mkdir(location_info_path)
if not os.path.exists(preprocessing_path):
    os.mkdir(preprocessing_path)

for file_name in file_list:
    # 파일의 날짜 형식이 22.01과 _2211 두가지로 존재한다.
    # .과 _를 제거하여 날짜형식을 2211과 같이 통일한다.
    save_file_name = (file_name[-9:]
                      .replace('.', '')
                      .replace('_', '')
                      .replace('csv', ''))
    print(save_file_name, file_name)

    # csv 처리를 한다.
    preprocess_csv(file_name, save_file_name, data_path)
print(f'A total of {len(file_list)} csv files were processed.')

### 3.2 "서울시 공공자전거 대여소" 전처리



"서울시 공공자전거 대여소"에 대한 데이터는 아래 과정을 통해 전처리를 수행하겠습니다. 

1. **세부 주소 통합**

    두 개의 열로 나누어져 있는 주소(메인주소와 서브주소)를 하나로 합쳐주겠습니다. 

2. **추가 정보 통합**

    "서울시 공공자전거 대여 이력 정보" 전처리를 수행하며 분할하였던 대여소 정보들을 불러와 "대여소ID"값을 기준으로 통합해 주겠습니다.

3. **위도와 경도 값의 이상치 제거**

    데이터 세트의 설명란에서 위도와 경도의 값이 0인 대여소(행)는 더 이상 사용되지 않는 대여소라고 하였기에 위도와 경도 값이 0인 행들을 찾아 모두 제거합니다.

4. **NA값 처리**

    거치대의 개수가 NA인 경우 0으로 값을 치환합니다.

5. **CSV로 저장**

    데이터 분석시 "대여소 정보"를 바로 사용할 수 있도록 CSV로 저장합니다.

#### 3.2.1 세부주소 통합



"대여소 정보.csv"에서 STATN_ADDR1는 메인 주소이고 STATN_ADDR2는 서브 주소입니다.

서브 주소의 경우 많은 Na값을 포함하고 있기에 Na값을 빈 문자열(`""`)로 치환한 뒤 두 열을 하나의 열로 통합하여 주겠습니다.


In [ ]:
location_path = 'data/대여소 정보.csv'
location = pd.read_csv(location_path)

# na값을 빈 값으로 채운 뒤
location = location.fillna('')

# STATN_ADDR1과 STATN_ADDR2 열의 값을 합쳐 하나의 열(주소)로 구성한다.
location = location.assign(주소=location.STATN_ADDR1 + ' ' + location.STATN_ADDR2)

# STATN_ADDR1과 STATN_ADDR2 열 삭제한다.
location = location.drop(columns=['STATN_ADDR1', 'STATN_ADDR2'])
location.head(5)

# Na값이 0개 인 것을 확인 할 수 있다.
print(f'Location data contain {location.isna().sum().sum()} na values')

#### 3.2.2 추가 정보 통합



"서울시 공공자전거 대여 이력" 정보의 전처리 과정에서 분할하였던 대여소에 대한 데이터를 "대여소ID"값을 기준으로 통합해 주겠습니다.

통합하는 과정은 아래와 같습니다.

1. locationInfo 폴더 내부에 저장된 대여소 정보들을 모두 읽어옵니다.

2. 대여소ID 열을 기준으로 중복으로 제거합니다.

3. "대여소 정보.csv"에서 대여소ID값을 기준으로 두 테이블을 결합합니다.

4. 열의 이름을 한글로 통일합니다.


In [ ]:
# 폴더의 경로를 설정한다.
common_table = pd.DataFrame(columns=['대여소ID', '대여소번호', '대여소명', '거치대'])

# 각 csv를 읽어와 대여소ID를 기준으로 중복된 열을 제거한다.
for file in os.listdir(location_info_path):
    file_path = os.path.join(location_info_path, file)
    if (os.path.splitext(file_path)[1] == '.csv'):
        table = pd.read_csv(file_path, encoding='UTF-8')
        if (len(common_table) == 0):
            common_table = table
        else:
            common_table = (pd.concat(
                [common_table, table], ignore_index=True).drop_duplicates(subset="대여소ID"))

# 대여소ID를 index로 설정한다.
common_table = common_table.set_index('대여소ID', drop=True)

# LENDPLACE_ID와 대여소ID값을 기준으로 LEFT JOIN을 수행한다.
location = pd.merge(location, common_table,
                    left_on='LENDPLACE_ID', right_on='대여소ID', how='left')

# 열의 이름을 한글로 통일해준다.
location = location.rename(
    columns={'LENDPLACE_ID': '대여소ID', 'STATN_LAT': '위도', 'STATN_LNT': '경도'})

# 필수적으로 필요한 열(대여소ID, 위도, 경도)은 Na값이 없는 것은 확인 할 수 있다.
print(
    f'Location data contain {location.iloc[:,0:3].isna().sum().sum()} na values')

# 올바르게 합쳐졌음을 확인 할 수 있다.
location.head(5)

#### 3.2.3 위도와 경도 값의 이상치 제거



데이터 세트의 설명란에서 위도와 경도의 값이 0인 대여소(행)는 더 이상 사용되지 않는 대여소라고 하였습니다.

따라서 위도와 경도 값이 0인 행들을 찾아 모두 제거합니다.


In [ ]:
# 경도와 위도의 좌표가 0인 값이 총 77개가 존재한다.
(location[['위도', '경도']] != 0.0).all(True).value_counts()

In [ ]:
# 경도와 위도의 좌표가 0인 행들을 모두 제거한다.
location = location[(location[['위도', '경도']] != 0.0).all(True)]
(location[['위도', '경도']] != 0.0).all(True).value_counts()

#### 3.2.4 NA값 처리



추가 정보를 통합한 결과 위도와 경도값이 0인 행에서 거치대의 개수가 Na인 것을 확인 할 수 있습니다.

해당값들의 존재 유무는 크게 영향을 받지 않기에 0으로 채워주겠습니다.


In [ ]:
# 거치대 열의 결측값을 채워준다.
location['거치대'] = location['거치대'].fillna(0)

#### 3.2.5 CSV로 저장



전처리가 완료된 값으로 "대여소 정보.csv"를 덮어씌우겠습니다.


In [ ]:
# csv로 저장한다.
location.to_csv(location_path, index=False, mode='w', encoding='UTF-8-sig')

### 3.3 "종관기상관측(ASOS)" 전처리



"종관기상관측(ASOS)"에 대한 데이터는 아래 과정을 통해 전처리를 수행하겠습니다. 

1. **가설 설정 및 정보 추출**

    "종관기상관측(ASOS)" 자료 중 가설을 설정하고 해당 가설에 해당하는 정보만을 추출하겠습니다.

2. **이상치 조정**

    이상치를 적절한 값으로 조정합니다. 

3. **CSV로 저장**

    데이터 분석시에 정보를 바로 사용할 수 있도록 CSV 파일로 저장합니다. 

#### 3.3.1 가설 설정 및 정보 추출



1.  **기온이 지나치게 낮거나 지나치게 높을 경우 공공자전거 이용율이 감소할 것이다.**

    전체 평균기온의 평균기온을 기준으로 차를 계산합니다.

2.  **일교차가 클 경우 공공자전거 이용률이 감소할 것이다.**

    최고기온에서 최저기온을 빼서 일교차 값을 구합니다.

3.  **비가 오면 공공자전거 이용률이 감소할 것이다.**

    '강수 계속시간(hr)' 값으로 강수시간과 공공자전거 이용률 간의 상관관계를 분석합니다.

    > 본 데이터 세트에서 강수와 공공자전거 이용률 간의 상관관계는 '일강수량(mm)' 값을 통해 분석하는 것이 더 타당한 것으로 판단됩니다. <br>하지만 '강수 계속시간(hr)'이 0 이상(비가 왔다)임에도 '일강수량(mm)' 값이 0인 오류를 내포하고 있으므로 '강수 계속시간(hr)'이 더 올바르게 측정된 자료라고 판단할 수 있습니다. <br>이에 '일 강수량(mm)' 대신 '강수 계속시간(hr)'값을 통해 강수와 공공자전거 이용률 간의 상관관계를 분석합니다.

4.  **눈이 오면 공공자전거 이용율이 감소할 것이다.**

    '일 최심적설(cm)' 값으로 적설량과 공공자전거 이용률 간의 상관관계를 분석합니다.

5.  **안개로 인하여 가시거리가 축소되면 공공자전거 이용률이 감소할 것이다.**

    '안개 계속시간(hr)' 값의 존재여부에 따라 안개가 발생하였다고 판단합니다.

    > 안개 또한 공공자전거 이용율에 영향을 미칠 것으로 판단되나 본 데이터 세트에서 '안개 계속시간(hr)'이 1개의 값만을 가지고 있으므로 유연한 판단이 어렵습니다. 따라서 **안개에 따른 영향은 고려하지 않습니다.**


In [ ]:
# 2022년 7월부터 2022년 12월 사이의 공공자전거 대여 이력 정보를 사용하기에
# 날짜를 인덱스로 하여 7월부터 12월 사이의 날짜를 가져온다.
weather = pd.read_csv(
    'data/seoul_weather_2022.csv',
    encoding='cp949',
    index_col="일시"
).loc['2022-07-01':'2022-12-31']


# 사용할 정보들을 담을 새 데이터프레임을 정의한다.
use_weather = pd.DataFrame(index=weather.index)

**1. 기온이 지나치게 낮거나 지나치게 높을 경우 공공자전거 이용율이 감소할 것이다**

전체 평균기온의 평균기온을 기준으로 차를 계산합니다.


In [ ]:
# 평균기온 열에 Na 값이 없는 것을 확인 할 수 있다.
weather['평균기온(°C)'].isna().sum()  # 0

mean_weather = weather['평균기온(°C)'].mean()
use_weather['temp_diff'] = abs(weather['평균기온(°C)'] - mean_weather)
use_weather.head(5).T

**2. 일교차가 클 경우 공공자전거 이용률이 감소할 것이다.**

최고기온에서 최저기온을 빼서 일교차 값을 구합니다.

최고기온에서 최저기온을 빼었을 때 NaN 값이 하나 존재합니다. 이는 최저기온의 2022-08-08 값이 NaN이기 때문입니다.

In [ ]:
gap = weather['최고기온(°C)'] - weather['최저기온(°C)']

# 최고기온에서 최저기온을 빼었을 때 NaN 값이 하나 존재한다.
print(f'Na count: {gap.isna().sum()}')

# 조사 결과 2022-08-08에서 최저기온 값이 NaN이다.
weather[((gap >= 0) == False)]

이에 2022-08-08의 값을 앞뒤 날의 최저기온 평균값으로 대치하겠습니다.

In [ ]:
# 2022-08-08의 값을 앞뒤날의 최저기온 평균값으로 대치한다.
gap['2022-08-08'] = (gap['2022-08-07'] + gap['2022-08-09']) / 2

# use_weather 데이터프레임에 일교차값을 추가한다.
use_weather['gap'] = gap
use_weather.head(5).T

**3. 비가 오면 공공자전거 이용률이 감소할 것이다.**

'강수 계속시간(hr)' 값으로 강수시간과 공공자전거 이용률 간의 상관관계를 분석합니다.

> 본 데이터 세트에서 강수와 공공자전거 이용률 간의 상관관계는 '일강수량(mm)' 값을 통해 분석하는 것이 더 타당한 것으로 판단됩니다. <br>하지만 '강수 계속시간(hr)'이 0 이상(비가 왔다)임에도 '일강수량(mm)' 값이 0인 오류를 내포하고 있음으로 '강수 계속시간(hr)'이 더 올바르게 측정된 자료라고 판단할 수 있습니다. <br>이에 '일강수량(mm)' 대신 '강수 계속시간(hr)'값을 통해 강수와 공공자전거 이용률 간의 상관관계를 분석합니다.


In [ ]:
# 비가 오지 않은 날은 Na로 처리되어 있다. 이를 0으로 변경하여 Na값을 처리한다.
# 강수 계속시간이 정상적인(시간임으로 0이상 24이하) 값의 분포를 가짐을 확인할 수 있다.
weather['강수 계속시간(hr)'].fillna(0).describe()

In [ ]:
# use_weather 데이터프레임에 강수 계속시간을 추가한다.
use_weather['rain_hr'] = weather['강수 계속시간(hr)'].fillna(0)
use_weather.head(5).T

**4. 눈이 오면 공공자전거 이용률이 감소할 것이다.**

'일 최심적설(cm)' 값으로 적설량과 공공자전거 이용률 간의 상관관계를 분석합니다.


In [ ]:
# 눈이 오지 않은 날은 Na로 처리되어 있다. 이를 0으로 변경하여 Na값을 처리한다.
weather['일 최심적설(cm)'].fillna(0).describe()

In [ ]:
use_weather['snow'] = weather['일 최심적설(cm)'].fillna(0)
use_weather.head(5).T

**생성된 데이터프레임 확인**

위 과정을 통해 생성된 use_weather 데이터프레임을 확인합니다.


In [ ]:
print(use_weather.head(5).T)
use_weather.describe().T

#### 3.3.2 이상치 조정



이상치를 적절하게 조정하여 지나치게 크거나 동떨어진 데이터가 데이터 분포에 영향을 주는 것을 방지합니다.


**1. 강수시간 이상치 조정**

위, use_weather 데이터프레임의 describe에서 강수시간(rain_hr)은 평균이 3 이하의 값을 가지는데 최댓값은 24임을 확인할 수 있습니다.

평균에서 지나치게 떨어진 값은 분포에 영향을 미칠 수 있음으로 적절한 임계값으로 조정해 주고자 합니다.

최댓값이 평균에 비해 지나치게 큰 값을 가지는지 알아보기 위해 데이터의 분포를 그래프로 표현해보겠습니다. 

In [ ]:
from matplotlib.pyplot import subplots

rain_hr = use_weather['rain_hr']

# 비가 오지 않은 날의 비율
print(f'Number of days without rain: {rain_hr[rain_hr == 0].count() / rain_hr.count() * 100:.2f}')

# 0보다 큰(초과) 값들 중 10 이상의 값을 가지는 비율은 약 20%이다.
print(f'Percentage of values greater than 0 that have a value greater than 10: {rain_hr[rain_hr > 10].count() / rain_hr[rain_hr != 0].count() * 100:.2f}')  # 30

# 강수계속시간을 반올림하여 그래프로 표현하였을 때 
# 10시간 이하의 값이 많으며(90% 이상) 10시간 이상의 값들은 각 시각에 따라 비교적 균등하게 분포되어 있음을 확인할 수 있다.
# 강수계속시간이 0(비가 오지 않았다)인 값의 크기가 비대하여 그래  프에 표시하지 않는다.
fig, ax = subplots(figsize=(14, 6))
pd.Series(rain_hr.values.round()).value_counts().sort_index()[1:].plot.bar()

위 분석을 통해 확인할 수 있듯 절반 이상의 값이 0(비가 오지 않았다)을 가지며 10 이상의 강수 계속시간은 비교적 균등한 분포를 이루고 있고 이를 통해 강수 계속시간의 평균이 3 이하의 값을 가지는데 최댓값은 24인 이유는 0(비가 오지 않았다)의 값이 지나치게 많기 때문이라는 것을 확인할 수 있습니다. 

따라서 본 현상을 **이상치** 라고 정의할 수 없고 이상치 조정을 수행할 수 없습니다.

>강수 계속시간이 24시간 이어진다는 점은 의심스러우나 명확한 증거가 없으며 10시간 이후 각 시간에 대한 데이터가 비교적 균등한 점을 고려하여 **이상치**로 정의하지 않았습니다.

#### 3.3.3 CSV로 저장



위 과정을 통해 생성된 use_weather 데이터프레임을 확인한 뒤, csv로 저장합니다.


In [ ]:
print(use_weather.head(5).T)
use_weather.describe().T

In [ ]:
use_weather.to_csv('data/weather.csv')
print('Save to csv: data/weather.csv')